In [8]:
page_link = 'https://subsidies.qoldau.kz/ru/subsidies/herbicides/pub-apps'
response = d.get(page_link)
print(d.execute_script("return navigator.userAgent"))

WebDriverException: Message: chrome not reachable
  (Session info: chrome=87.0.4280.66)


In [17]:
# LIBRARIES
# *********
import requests # request for the web pagegg
from fake_useragent import UserAgent # to fake meta data
from requests.packages.urllib3.exceptions import InsecureRequestWarning # handle while requesting for the web page
from selenium import webdriver # create a simulation of the pseudo-user
import time # set sleep on web page on every click, handling nginx error
import datetime  # record start time
from bs4 import BeautifulSoup # to get web page soure
import pandas as pd # create df
import numpy as np # manipulate with shape of the df
import pyodbc
import csv # get rid off quotes
import json
lol = lambda lst, sz: [lst[i:i+sz] for i in range(0, len(lst), sz)]

In [18]:
options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
options.add_argument('disable-infobars')
d = webdriver.Chrome(chrome_options=options, executable_path=r"\\aafssrv\SHARED-BI\driver\chromedriver.exe")

<ipython-input-18-d89da3cdb56e>:4: DeprecationWarning: use options instead of chrome_options
  d = webdriver.Chrome(chrome_options=options, executable_path=r"\\aafssrv\SHARED-BI\driver\chromedriver.exe")


applications

In [ ]:
# L I N K S
# baseURL of Qoldau
baseUrl = "https://subsidies.qoldau.kz/ru/subsidies/"

# applications
appFerti = "fertilizer/pub-apps"
appPest = "herbicides/pub-apps"
appSeed = "seed/pub-apps"
y2019 = "?Year=2019"

listOfLinks = [appPest, appFerti, appPest+y2019, appFerti+y2019] #appSeed,  
listOfNames =  [ "Пестициды2020", "Удобрения2020", "Пестициды2019", "Удобрения2019"] #"Семена2019_2020", 

linkN = 0
for link in listOfLinks:
    print(linkN, " = ", listOfNames[linkN])
    print("Started: ", datetime.datetime.now().astimezone().strftime("%Y-%m-%d, %H:%M:%S"))
    d.get(baseUrl+link)#, headers={'User-Agent': UserAgent().chrome})
    time.sleep(2)
    html = d.page_source
    nPages = int(d.find_elements_by_xpath('//li[@data-page-index]/a')[-2].text)
    print("This page's Navgation steps = ", nPages)
    tempLastN = []
    tempRowData = []
    cols = d.find_element_by_xpath("//div[@class='sw-result-table-container']/div/div/table").find_elements_by_tag_name('th')
    colNames = []
    tempColNames = []
    for col in cols:
        r = col.get_attribute("innerText").replace('\n', ',')
        tempColNames.append(r)
    colN = len(tempColNames)
    print("Number of cols = ", colN, "Columns are ", tempColNames)
    for LastN in d.find_elements_by_xpath("//div[@class='pull-right']/div/span"):
        tempLastN.append(LastN.text.split(' '))
    lastN = int(tempLastN[-1][-1])
    for click in range(nPages):
        rows = d.find_element_by_xpath("//div[@class='sw-result-table-container']/div/div/table/tbody").find_elements_by_tag_name('td')
        if(rows[0].text != "Нет записей"):
            for row in rows:
                tempRowData.append(row.text)
            d.find_elements_by_xpath('//li[@data-page-index]/a')[-1].click()
            time.sleep(3)
        count = 0
    f = pd.DataFrame(lol(tempRowData, colN), columns=tempColNames)
    f.to_csv(r'\\aafssrv\SHARED-BI\files_aa_dwh_x\dataBase\qldData\apps\/' + listOfNames[linkN] + '.csv', encoding='utf-8', sep='|', quoting = csv.QUOTE_NONE, quotechar="",  escapechar="~")
    linkN += 1
    tempRowData = []
    print("Finished: ", datetime.datetime.now().astimezone().strftime("%Y-%m-%d, %H:%M:%S"))

0  =  Пестициды2020
Started:  2020-12-01, 17:45:50
This page's Navgation steps =  1688
Number of cols =  16 Columns are  ['№', 'Область', 'Дата подачи заявки', 'Наименование заявителя', 'Производитель пестицидов', 'Наименование препарата', 'Объем пестицида в базовой ед.измерения', 'Базовая единица измерения', 'Площадь использования, га', 'Статус', 'Тип заявки', 'Дата принятия заявки', 'Дата отклонения', 'Сумма субсидий (расчетная), тг.', 'Причина отклонения', 'Примечание к отрицательному решению']
Finished:  2020-12-02, 01:40:34
1  =  Удобрения2020
Started:  2020-12-02, 01:40:34
This page's Navgation steps =  953
Number of cols =  16 Columns are  ['№', 'Область', 'Наименование заявителя', 'Дата подачи заявки', 'Статус', 'Площадь использования, га', 'Производитель удобрений', 'Наименование удобрения', 'Единица измерения', 'Объем удобрения', 'Сумма субсидий (расчетная), тг.', 'Тип заявки', 'Дата принятия заявки', 'Дата отрицательного решения', 'Примечание к отрицательному решению', 'Прич

справочники - Удобр

In [12]:
# L I N K S
# ************************************************************************************
# baseURL of Qoldau
baseUrl = "https://subsidies.qoldau.kz/ru/subsidies/"
# groups
ferti = "fertilizer/"
pest = "herbicides/"
seed = "seed/"
# dicts of prods
subsNormsOfCost = "refs/subsidies-norms/"
sprUsageNorms = "refs/usage-norms/"
sprSeed = "seeds"

###    НОРМЫ ИСПОЛЬЗОВАНИЯ УДОБРЕНИЙ 
AKM = "94/culutre"
AKT = "91/culutre"
ALM = "84/culutre"
ATR = "99/culutre"
ZKO = "88/culutre"
ZHML = "87/culutre"
KRG = "90/culutre"
KST = "85/culutre"
KZL = "92/culutre"
MNG = "98/culutre"
TUR = "96/culutre"
PVL = "97/culutre"
SKO = "86/culutre"
VKO = "89/culutre"
SHYM = "100/culutre"

listOfLinks = [pest+sprUsageNorms, seed+sprSeed, ferti+sprUsageNorms+AKM, ferti+sprUsageNorms+AKT, ferti+sprUsageNorms+ALM, ferti+sprUsageNorms+ATR, 
               ferti+sprUsageNorms+ZKO, ferti+sprUsageNorms+ZHML, ferti+sprUsageNorms+KRG, 
               ferti+sprUsageNorms+KST, ferti+sprUsageNorms+KZL, ferti+sprUsageNorms+MNG, 
               ferti+sprUsageNorms+TUR, ferti+sprUsageNorms+PVL, ferti+sprUsageNorms+SKO, 
               ferti+sprUsageNorms+VKO, ferti+sprUsageNorms+SHYM]
listOfNames = ["pestUsageNorms", "seedsDim", "fertiUsageNormsAKM", "fertiUsageNormsAKT", "fertiUsageNormsALM", "fertiUsageNormsATR", "fertiUsageNormsZKO", "fertiUsageNormsZHML", "fertiUsageNormsKRG", "fertiUsageNormsKST", "fertiUsageNormsKZL", "fertiUsageNormsMNG", "fertiUsageNormsTUR", "fertiUsageNormsPVL", "fertiUsageNormsSKO", "fertiUsageNormsVKO", "fertiUsageNormsSHYM"]

linkN = 0
all_f = pd.DataFrame()
for link in listOfLinks:
    print(linkN, " = ", listOfNames[linkN])
    print("Started: ", datetime.datetime.now().astimezone().strftime("%Y-%m-%d, %H:%M:%S"))
    d.get(baseUrl+link)
    time.sleep(2)
    html = d.page_source
    nPages = int(d.find_elements_by_xpath('//li[@data-page-index]/a')[-2].text)
    print("This page's Navgation steps = ", nPages)
    tempLastN = []
    tempRowData = []
    cols = d.find_element_by_xpath("//div[@class='sw-result-table-container']/div/div/table").find_elements_by_tag_name('th')
    colNames = []
    tempColNames = []
    for col in cols:
        r = col.get_attribute("innerText").replace('\n', ',')
        tempColNames.append(r)
    colN = len(tempColNames)
    print("Number of cols = ", colN, "Columns are ", tempColNames)
    for LastN in d.find_elements_by_xpath("//div[@class='pull-right']/div/span"):
        tempLastN.append(LastN.text.split(' '))
    lastN = int(tempLastN[-1][-1])
    for click in range(nPages):
        rows = d.find_element_by_xpath("//div[@class='sw-result-table-container']/div/div/table/tbody").find_elements_by_tag_name('td')
        if(rows[0].text != "Нет записей"):
            for row in rows:
                tempRowData.append(row.text)
            d.find_elements_by_xpath('//li[@data-page-index]/a')[-1].click()
            time.sleep(2)
        count = 0
        testData = np.reshape(tempRowData, (-1, colN))
    f = pd.DataFrame(testData, columns=tempColNames)
    try:
        region = d.find_elements_by_xpath("/html/body/div[2]/div/div[2]/div[2]/div[1]/div[2]/form/div[2]/div/div[2]/div/span")[0].text
        f["region"] = region
        tempRowData = []
        all_f = all_f.append(f, ignore_index=True)
        all_f.to_csv(r'\\aafssrv\SHARED-BI\files_aa_dwh_x\dataBase\qldData\dim\fertiUsageNorms.csv', encoding='utf-8', sep='|', quoting = csv.QUOTE_NONE, escapechar = '')
        linkN += 1
    except IndexError:
        tempRowData = []  
        print("Finished: ", datetime.datetime.now().astimezone().strftime("%Y-%m-%d, %H:%M:%S"))
        f.to_csv(r'\\aafssrv\SHARED-BI\files_aa_dwh_x\dataBase\qldData\dim\/' + listOfNames[linkN] +  '.csv', encoding='utf-8', sep='|', quoting = csv.QUOTE_NONE, quotechar="",  escapechar="~")
        linkN += 1

0  =  pestUsageNorms
Started:  2020-12-01, 04:29:35
This page's Navgation steps =  249
Number of cols =  8 Columns are  ['№', 'Культура', 'Наименование препарата', 'Сорняк', 'Ед. измерения', 'Норма применения с', 'Норма применения по', 'Дата окончания срока регистрации (число, месяц, год)']
Finished:  2020-12-01, 05:05:38
1  =  seedsDim
Started:  2020-12-01, 05:05:38
This page's Navgation steps =  312
Number of cols =  7 Columns are  ['№', 'Наименование', 'Вид (рус)', 'Вид (лат)', 'Репродукция', 'Тип', 'Культура']
Finished:  2020-12-01, 05:44:50
2  =  fertiUsageNormsAKM
Started:  2020-12-01, 05:44:50
This page's Navgation steps =  449
Number of cols =  6 Columns are  ['№', 'Наименование культуры', 'Наименование удобрения', 'Ед. измерения', 'Норма применения', 'Тип орошения']
3  =  fertiUsageNormsAKT
Started:  2020-12-01, 06:36:20
This page's Navgation steps =  84
Number of cols =  6 Columns are  ['№', 'Наименование культуры', 'Наименование удобрения', 'Ед. измерения', 'Норма применения

Norm of costs

In [15]:
# L I N K S
# ************************************************************************************
# groups
ferti = "fertilizer/"
pest = "herbicides/"
# dicts of prods
subsNormsOfCost = "refs/subsidies-norms/"
sprUsageNorms = "refs/usage-norms/"
sprSeed = "seeds"

###    НОРМЫ СУБСИДИРОВАНИЯ СТОИМОСТИ 
AKM = "94/year/"
AKT = "91/year/"
ALM = "84/year/"
ATR = "99/year/"
ZKO = "88/year/"
ZHML = "87/year/"
KRG = "90/year/"
KST = "85/year/"
KZL = "92/year/"
MNG = "98/year/"
TUR = "96/year/"
PVL = "97/year/"
SKO = "86/year/"
VKO = "89/year/"
SHYM = "100/year/"

fertiListOfLinks = [ferti+subsNormsOfCost+AKM, ferti+subsNormsOfCost+AKT, ferti+subsNormsOfCost+ALM, ferti+subsNormsOfCost+ATR, ferti+subsNormsOfCost+ZKO, ferti+subsNormsOfCost+ZHML, ferti+subsNormsOfCost+KRG, ferti+subsNormsOfCost+KST, ferti+subsNormsOfCost+KZL, ferti+subsNormsOfCost+MNG, ferti+subsNormsOfCost+TUR, ferti+subsNormsOfCost+PVL, ferti+subsNormsOfCost+SKO, ferti+subsNormsOfCost+VKO, ferti+subsNormsOfCost+SHYM]
fertiListOfNames = ["fertiSubsNormsOfCostAKT", "fertiSubsNormsOfCostALM", "fertiSubsNormsOfCostATR", "fertiSubsNormsOfCostZKO", "fertiSubsNormsOfCostZHML", "fertiSubsNormsOfCostKRG", "fertiSubsNormsOfCostKST", "fertiSubsNormsOfCostKZL", "fertiSubsNormsOfCostMNG", "fertiSubsNormsOfCostTUR", "fertiSubsNormsOfCostPVL", "fertiSubsNormsOfCostSKO", "fertiSubsNormsOfCostVKO", "fertiSubsNormsOfCostSHYM"]

pestListOfLinks = [pest+subsNormsOfCost+AKM, pest+subsNormsOfCost+AKT, pest+subsNormsOfCost+ALM, pest+subsNormsOfCost+ATR, pest+subsNormsOfCost+ZKO, pest+subsNormsOfCost+ZHML, pest+subsNormsOfCost+KRG, pest+subsNormsOfCost+KST, pest+subsNormsOfCost+KZL, pest+subsNormsOfCost+MNG, pest+subsNormsOfCost+TUR, pest+subsNormsOfCost+PVL, pest+subsNormsOfCost+SKO, pest+subsNormsOfCost+VKO, pest+subsNormsOfCost+SHYM]
pestListOfNames = ["pestSubsNormsOfCostAKT", "pestSubsNormsOfCostALM", "pestSubsNormsOfCostATR", "pestSubsNormsOfCostZKO", "pestSubsNormsOfCostZHML", "pestSubsNormsOfCostKRG", "pestSubsNormsOfCostKST", "pestSubsNormsOfCostKZL", "pestSubsNormsOfCostMNG", "pestSubsNormsOfCostTUR", "pestSubsNormsOfCostPVL", "pestSubsNormsOfCostSKO", "pestSubsNormsOfCostVKO", "pestSubsNormsOfCostSHYM"]


def subsNormsOfCostOfProd(listOfLinks, category, year):
    baseUrl = "https://subsidies.qoldau.kz/ru/subsidies/"
    baseUrl = baseUrl
    allLinks = listOfLinks
    category = category
    linkN = 0
    all_f = pd.DataFrame()
    for link in allLinks:
        print(linkN, " = ", listOfNames[linkN])
        print("Started: ", datetime.datetime.now().astimezone().strftime("%Y-%m-%d, %H:%M:%S"))
        d.get(baseUrl+link+year)
        time.sleep(1)
        html = d.page_source
        nPages = int(d.find_elements_by_xpath('//li[@data-page-index]/a')[-2].text)
        print("This page's Navgation steps = ", nPages)
        tempLastN = []
        tempRowData = []
        cols = d.find_element_by_xpath("//div[@class='sw-result-table-container']/div/div/table").find_elements_by_tag_name('th')
        colNames = []
        tempColNames = []
        for col in cols:
            r = col.get_attribute("innerText").replace('\n', ',')
            time.sleep(1)
            tempColNames.append(r)
        colN = len(tempColNames)
        print("Number of cols = ", colN, "Columns are ", tempColNames)
        for LastN in d.find_elements_by_xpath("//div[@class='pull-right']/div/span"):
            tempLastN.append(LastN.text.split(' '))
        lastN = int(tempLastN[-1][-1])
        for click in range(nPages):
            rows = d.find_element_by_xpath("//div[@class='sw-result-table-container']/div/div/table/tbody").find_elements_by_tag_name('td')
            if(rows[0].text != "Нет записей"):
                for row in rows:
                    tempRowData.append(row.text)
                d.find_elements_by_xpath('//li[@data-page-index]/a')[-1].click()
                time.sleep(1)
            count = 0
            testData = np.reshape(tempRowData, (-1, colN))
        f = pd.DataFrame(testData, columns=tempColNames)
        region = d.find_elements_by_xpath("/html/body/div[2]/div/div[2]/div[2]/div[1]/div[2]/form/div[2]/div/div[2]/div/span")[0].text
        f["region"] = region
        f["group"] = category
        f["year"] = str(year)
        tempRowData = []
        all_f = all_f.append(f, ignore_index=True)
        all_f.to_csv(r'\\aafssrv\SHARED-BI\files_aa_dwh_x\dataBase\qldData\costs\/'+category+'_subsNormsOfCostOfProd_'+str(year)+'.csv', encoding='utf-8', sep='|', quoting = csv.QUOTE_NONE, quotechar="",  escapechar="~")
        linkN += 1 
        print("Finished: ", datetime.datetime.now().astimezone().strftime("%Y-%m-%d, %H:%M:%S"))
        
subsNormsOfCostOfProd(fertiListOfLinks, "Удобрения","2019")
subsNormsOfCostOfProd(fertiListOfLinks, "Удобрения","2020")

subsNormsOfCostOfProd(fertiListOfLinks, "Пестициды","2019")
subsNormsOfCostOfProd(fertiListOfLinks, "Пестициды","2020")

0  =  pestUsageNorms
Started:  2020-12-01, 15:27:15
This page's Navgation steps =  15
Number of cols =  4 Columns are  ['№', 'Наименование удобрения', 'Ед.измерения', 'Норма субсидий, тг.']
Finished:  2020-12-01, 15:28:25
1  =  seedsDim
Started:  2020-12-01, 15:28:25
This page's Navgation steps =  15
Number of cols =  4 Columns are  ['№', 'Наименование удобрения', 'Ед.измерения', 'Норма субсидий, тг.']
Finished:  2020-12-01, 15:29:26
2  =  fertiUsageNormsAKM
Started:  2020-12-01, 15:29:26
This page's Navgation steps =  15
Number of cols =  4 Columns are  ['№', 'Наименование удобрения', 'Ед.измерения', 'Норма субсидий, тг.']
Finished:  2020-12-01, 15:30:31
3  =  fertiUsageNormsAKT
Started:  2020-12-01, 15:30:31
This page's Navgation steps =  15
Number of cols =  4 Columns are  ['№', 'Наименование удобрения', 'Ед.измерения', 'Норма субсидий, тг.']
Finished:  2020-12-01, 15:31:35
4  =  fertiUsageNormsALM
Started:  2020-12-01, 15:31:35
This page's Navgation steps =  15
Number of cols =  4 

In [16]:
baseColNames  = ['Область', 'ДатаПодачиЗаявки', 'Заявитель',
                 'Поставщик', 'Продукт',
                 'Объем', 'БазоваяЕдиницаИзмерения',
                 'ПлощадьИспользованияГа', 'Статус', 'ТипЗаявки',
                 'ДатаПринятияЗаявки', 'ДатаОтклонения',
                 'СуммаСубсидийТг', 'ПричинаОтклонения',
                 'ПримечаниеКОтрицательномуРешению', 'group', 'year']
      

#---------------------------------------------------------------------------------
baseColNamesReorderForFerti = ['Область', 'Дата подачи заявки', 'Наименование заявителя',
                        'Производитель удобрений', 'Наименование удобрения', 
                        'Объем удобрения', 'Единица измерения',
                        'Площадь использования, га', 'Статус', 'Тип заявки',
                        'Дата принятия заявки', 'Дата отрицательного решения', 
                        'Сумма субсидий (расчетная), тг.', 'Причина отрицательного решения',
                        'Примечание к отрицательному решению', 'group', 'year']

#---------------------------------------------------------------------------------
baseColNamesReorderForSeeds = ['Область', 'Дата подачи заявки', 'Наименование заявителя',
                        'Поставщик семян', 'Наименование', 'Объем семян в базовой ед.измерения',
                        'Базовая единица измерения', 'Площадь использования, га', 'Статус',
                        'Тип заявки', 'Дата принятия заявки', 'Дата отклонения',
                        'Сумма субсидий (расчетная), тг.', 'Причина отклонения',
                        'Примечание к отрицательному решению', 'group']


#------------------------------------ P E S T I C I D E S ---------------------------------------
p20 =  pd.read_csv(r'\\aafssrv\SHARED-BI\files_aa_dwh_x\dataBase\qldData\apps\Пестициды2020.csv', encoding='utf-8', sep='|')
p20['group'] = "Пестициды"
p20 = p20.iloc[:, 2:]
p20['year'] = "2020"
# print(p20.columns)
print(len(p20.columns))


p19 =  pd.read_csv(r'\\aafssrv\SHARED-BI\files_aa_dwh_x\dataBase\qldData\apps\Пестициды2019.csv', encoding='utf-8', sep='|')
p19['group'] = "Пестициды"
p19 = p19.iloc[:, 2:]
p19['year'] = "2019"
# print(p19.columns)
print(len(p19.columns))


p = pd.concat([p19, p20], ignore_index = True)
p.columns = baseColNames

#------------------------------------ F E R T L I Z E R S ---------------------------------------

f20 =  pd.read_csv(r'\\aafssrv\SHARED-BI\files_aa_dwh_x\dataBase\qldData\apps\Удобрения2020.csv', encoding='utf-8', sep='|')
f20['group'] = "Удобрения"
f20 = f20.iloc[:, 2:]
f20['year'] = "2020"

f19 =  pd.read_csv(r'\\aafssrv\SHARED-BI\files_aa_dwh_x\dataBase\qldData\apps\Удобрения2019.csv', encoding='utf-8', sep='|')
f19['group'] = "Удобрения"
f19 = f19.iloc[:, 2:]
f19['year'] = "2019"

f = pd.concat([f19, f20], ignore_index = True)
f = f[baseColNamesReorderForFerti]
f.columns = baseColNames

#------------------------------------ S E E D S ------------------------------------------------
sAll =  pd.read_csv(r'\\aafssrv\SHARED-BI\files_aa_dwh_x\dataBase\qldData\apps\Семена2019_2020.csv', encoding='utf-8', sep='|')
sAll['group'] = "Семена"
sAll = sAll.iloc[:, 2:]
sAll = sAll[baseColNamesReorderForSeeds]

s19 = sAll[sAll["Дата принятия заявки"].astype(str).str.contains("2019", case = False)]
s19['year'] = "2019"
s20 = sAll[sAll["Дата принятия заявки"].astype(str).str.contains("2020", case = False)]
s20['year'] = "2020"

s = pd.concat([s19, s20], ignore_index = True)
s.columns = baseColNames
#------------------------------------ A L L -- C A T E G O R I E S ------------------------------------------------
q = pd.concat([p, f, s], ignore_index = True)
q['applicantID'] =  q[['Заявитель', 'Область']].agg('-'.join, axis=1)
q = q.replace(r'\n',' ', regex=True)
q = q.replace(r'~',' ', regex=True)


#------------------------------------ C O N S T R A C T I O N S ------------------------------------------------
colsToUpper = [2,3]
for colUpper in range(len(colsToUpper)):
    q.iloc[:, colsToUpper[colUpper]] = q.iloc[:, colsToUpper[colUpper]].str.upper()
q = q.replace(to_replace = ["ТОВАРИЩЕСТВА С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ", "ТОВАРИЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ",
                            'ЖАУАПКЕРШІЛІГІ ШЕКТЕУЛІ СЕРІКТЕСТІГІ'], 
              value = "ТОО",
              regex=True)
q['Заявитель'] = q['Заявитель'].replace(to_replace = ['"', "'", '«', '»', '/', '\(', '\)'], 
              value = "", regex=True)
q['Поставщик'] = q['Поставщик'].replace(to_replace = ['"', "'", '«', '»', '/', '\(', '\)'], 
              value = "", regex=True)
q['Область'] = q['Область'].replace("Шымкент г.а.",  "г.Шымкент", regex=True)
q['Область'] = q['Область'].replace("Алматы г.а.",  "г.Алматы", regex=True)
q['Область'] = q['Область'].replace("Нур-Султан г.а.",  "г.Нур-Султан", regex=True)

#------------------------------------ D I S T R I B U T O R S ------------------------------------------------
pak = ['ТОО ПОЛГАР АГРОКЕМ', 'ТОО ПЕРВАЯ АГРОХИМИЧЕСКАЯ КОМПАНИЯ']
q['Поставщик'].replace({pak[0]: 'ПАК', pak[1]: 'ПАК', 'ТОО ALEM AGRO TRADE': 'ALEM AGRO'}, inplace=True, regex=True)
q['Поставщик'] = q['Поставщик'].str.replace('ТОО ALEM AGRO HOLDING АЛЕМАГРО ХОЛДИНГ', 'ALEM AGRO')

#------------------------------------  A A   l o g i c a l  c o l u m n ------------------------------------------------- 
q['АА'] = np.where(q['Поставщик'] == 'ALEM AGRO', 'Да', 'Нет')
aaClient = q[['АА', 'applicantID']].loc[q['АА'] == "Да"].drop_duplicates().reset_index().iloc[:,2:]
q['isSubsAaClient'] = q.applicantID.isin(aaClient.applicantID).astype(bool)

#------------------------------------ P A T H  ------------------------------------
q.to_csv(r'\\aafssrv\SHARED-BI\files_aa_dwh_x\dataBase\qldData\apps\q.csv', encoding='utf-8', sep='|', quoting = csv.QUOTE_NONE, quotechar="",  escapechar="~")
#------------------------------------ R A N D O M   5   R O W S ------------------------------------
q.sample(5)

17
17


KeyError: "['Область'] not in index"

In [ ]:
server = '192.168.1.209'  #tcp:myserver.database.windows.net 
database = 'AA_DWH_X' #mydb
username = 'pbi_report' #myusername 
password = '1s22s22p6' #mypassword
conn = pyodbc.connect('DRIVER={sql server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = conn.cursor()

In [ ]:
sql_1 = ('SELECT * FROM information_schema.tables')   #
df_1 = pd.read_sql(sql_1,conn)
df_1

In [ ]:
qName_sql = ('SELECT * FROM qName')
qName = pd.read_sql(qName_sql, conn)
qName = qName.applymap(str)
qName.codERP = qName.codERP.s tr.pad(9, side='left', fillchar='0')
qName

In [ ]:
cato_sql = ('SELECT * FROM cato')
cato = pd.read_sql(cato_sql, conn)
cato = cato.applymap(str)
# cato[cato.name.str.contains("Шымкент", case = False)]
cato
#г.Алматы г.Нур-Султан г.Шымкент

In [ ]:
qWithProdCodes = pd.merge(q, qName[["name", "codERP"]], left_on = "Продукт", right_on = "name", how='left').drop(columns = ["name"])
qWithKatoCodes = pd.merge(qWithProdCodes, cato[["name", "cato_Id"]], left_on = "Область", right_on = "name", how='left').drop(columns = ["name"])

In [ ]:
test = pd.melt(qWithKatoCodes.reset_index(), id_vars = ['index']).sort_values(by = ['index'])
test.to_csv(r"\\aafssrv\SHARED-BI\files_aa_dwh_x\testQoldau.csv", header = ['index1', 'variable', 'value'], encoding='utf-8', sep='|', quoting = csv.QUOTE_NONE, escapechar = '')

In [ ]:
baseColNames  = ['region', 'appDate', 'applicant',
                 'provider', 'product',
                 'volume', 'unit',
                 'areaOfUsage', 'status', 'typeOfApp',
                 'AppAcceptanceDate', 'DateOfRejection',
                 'subsidies', 'reasonOfRejection',
                 'NoteOnNegativeResolution', 'group', 'year']

#---------------------------------------------------------------------------------
baseColNamesReorderForFerti = ['Область', 'Дата подачи заявки', 'Наименование заявителя',
                        'Производитель удобрений', 'Наименование удобрения', 
                        'Объем удобрения', 'Единица измерения',
                        'Площадь использования, га', 'Статус', 'Тип заявки',
                        'Дата принятия заявки', 'Дата отрицательного решения', 
                        'Сумма субсидий (расчетная), тг.', 'Причина отрицательного решения',
                        'Примечание к отрицательному решению', 'group', 'year']

#---------------------------------------------------------------------------------
baseColNamesReorderForSeeds = ['Область', 'Дата подачи заявки', 'Наименование заявителя',
                        'Поставщик семян', 'Наименование', 'Объем семян в базовой ед.измерения',
                        'Базовая единица измерения', 'Площадь использования, га', 'Статус',
                        'Тип заявки', 'Дата принятия заявки', 'Дата отклонения',
                        'Сумма субсидий (расчетная), тг.', 'Причина отклонения',
                        'Примечание к отрицательному решению', 'group']


#------------------------------------ P E S T I C I D E S ---------------------------------------
p20 =  pd.read_csv(r'\\aafssrv\SHARED-BI\files_aa_dwh_x\dataBase\qldData\apps\Пестициды2020.csv', encoding='utf-8', sep='|')
p20['group'] = "Пестициды"
p20 = p20.iloc[:, 2:]
p20['year'] = "2020"
# print(p20.columns)
print(len(p20.columns))


p19 =  pd.read_csv(r'\\aafssrv\SHARED-BI\files_aa_dwh_x\dataBase\qldData\apps\Пестициды2019.csv', encoding='utf-8', sep='|')
p19['group'] = "Пестициды"
p19 = p19.iloc[:, 2:]
p19['year'] = "2019"
# print(p19.columns)
print(len(p19.columns))


p = pd.concat([p19, p20], ignore_index = True)
p.columns = baseColNames

#------------------------------------ F E R T L I Z E R S ---------------------------------------

f20 =  pd.read_csv(r'\\aafssrv\SHARED-BI\files_aa_dwh_x\dataBase\qldData\apps\Удобрения2020.csv', encoding='utf-8', sep='|')
f20['group'] = "Удобрения"
f20 = f20.iloc[:, 2:]
f20['year'] = "2020"

f19 =  pd.read_csv(r'\\aafssrv\SHARED-BI\files_aa_dwh_x\dataBase\qldData\apps\Удобрения2019.csv', encoding='utf-8', sep='|')
f19['group'] = "Удобрения"
f19 = f19.iloc[:, 2:]
f19['year'] = "2019"

f = pd.concat([f19, f20], ignore_index = True)
f = f[baseColNamesReorderForFerti]
f.columns = baseColNames

#------------------------------------ S E E D S ------------------------------------------------
sAll =  pd.read_csv(r'\\aafssrv\SHARED-BI\files_aa_dwh_x\dataBase\qldData\apps\Семена2019_2020.csv', encoding='utf-8', sep='|')
sAll['group'] = "Семена"
sAll = sAll.iloc[:, 2:]
sAll = sAll[baseColNamesReorderForSeeds]

s19 = sAll[sAll["Дата принятия заявки"].astype(str).str.contains("2019", case = False)]
s19['year'] = "2019"
s20 = sAll[sAll["Дата принятия заявки"].astype(str).str.contains("2020", case = False)]
s20['year'] = "2020"

s = pd.concat([s19, s20], ignore_index = True)
s.columns = baseColNames
#------------------------------------ A L L -- C A T E G O R I E S ------------------------------------------------
q = pd.concat([p, f, s], ignore_index = True)
q['applicantID'] =  q[['applicant', 'region']].agg('-'.join, axis=1)
q = q.replace(r'\n',' ', regex=True)
q = q.replace(r'~',' ', regex=True)
#------------------------------------ C O N S T R A C T I O N S ------------------------------------------------
colsToUpper = [2,3]
for colUpper in range(len(colsToUpper)):
    q.iloc[:, colsToUpper[colUpper]] = q.iloc[:, colsToUpper[colUpper]].str.upper()
q = q.replace(to_replace = ["ТОВАРИЩЕСТВА С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ", "ТОВАРИЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ",
                            'ЖАУАПКЕРШІЛІГІ ШЕКТЕУЛІ СЕРІКТЕСТІГІ'], 
              value = "ТОО",
              regex=True)
q['applicant'] = q['applicant'].replace(to_replace = ['"', "'", '«', '»', '/', '\(', '\)'], 
              value = "", regex=True)
q['provider'] = q['provider'].replace(to_replace = ['"', "'", '«', '»', '/', '\(', '\)'], 
              value = "", regex=True)
q['region'] = q['region'].replace("Шымкент г.а.",  "г.Шымкент", regex=True)
q['region'] = q['region'].replace("Алматы г.а.",  "г.Алматы", regex=True)
q['region'] = q['region'].replace("Нур-Султан г.а.",  "г.Нур-Султан", regex=True)


#------------------------------------ D I S T R I B U T O R S ------------------------------------------------
pak = ['ТОО ПОЛГАР АГРОКЕМ', 'ТОО ПЕРВАЯ АГРОХИМИЧЕСКАЯ КОМПАНИЯ']
q['provider'].replace({pak[0]: 'ПАК', pak[1]: 'ПАК', 'ТОО ALEM AGRO TRADE': 'ALEM AGRO'}, inplace=True, regex=True)
q['provider'] = q['provider'].str.replace('ТОО ALEM AGRO HOLDING АЛЕМАГРО ХОЛДИНГ', 'ALEM AGRO')
#------------------------------------  A A   l o g i c a l  c o l u m n ------------------------------------------------- 
q['АА'] = np.where(q['provider'] == 'ALEM AGRO', 'Да', 'Нет')
aaClient = q[['АА', 'applicantID']].loc[q['АА'] == "Да"].drop_duplicates().reset_index().iloc[:,2:]
q['isSubsAaClient'] = q.applicantID.isin(aaClient.applicantID).astype(bool)

#------------------------------------ P A T H  ---
q.to_csv(r'\\aafssrv\SHARED-BI\files_aa_dwh_x\dataBase\qldData\q_02.csv', encoding='utf-8', sep='|', quoting = csv.QUOTE_NONE, quotechar="",  escapechar="~")
#------------------------------------ R A N D O M   5   R O W S ------------------------------------
q.sample(5)

In [ ]:
qWithProdCodes = pd.merge(q, qName[["name", "codERP"]], left_on = "product", right_on = "name", how='left').drop(columns = ["name"])
qWithKatoCodes = pd.merge(qWithProdCodes, cato[["name", "cato_Id"]], left_on = "region", right_on = "name", how='left').drop(columns = ["name"])
TO_CHECK_Q = costNormsCodes_prod[qWithKatoCodes["codERP"].isna()]
TO_CHECK.to_excel(r"\\aafssrv\SHARED-BI\files_aa_dwh_x\dataBase\qldData\CHECK\TO_CHECK_Q.xlsx")

In [ ]:
test = pd.melt(qWithKatoCodes.reset_index(), id_vars = ['index']).sort_values(by = ['index'])
test.to_csv(r"\\aafssrv\SHARED-BI\files_aa_dwh_x\testQoldau_02.csv", header = ['index1', 'variable', 'value'], encoding='utf-8', sep='|', quoting = csv.QUOTE_NONE, quotechar="",  escapechar="~")

In [ ]:
testToInsert = pd.read_csv(r"\\aafssrv\SHARED-BI\files_aa_dwh_x\testQoldau_02.csv", encoding="utf-8", sep="|")
testToInsert = testToInsert.replace('~', '')
testToInsert.columns = ['index0', 'index1', 'variable', 'value']

In [ ]:
cursor.execute("DELETE FROM subsidies_02")
conn.commit()

cursor.fast_executemany = True
testQuery = "INSERT INTO subsidies_02(index0, index1, variable, value) VALUES (?, ?, ?, ?)"
cursor.executemany(testQuery, testToInsert.to_numpy().tolist())   
conn.commit()

In [ ]:
subsidies_02_sql = ('SELECT * FROM subsidies_02')
subsidies_02 = pd.read_sql(subsidies_02_sql, conn)
subsidies_02 = subsidies_02.applymap(str)
subsidies_02#.columns

In [ ]:
import glob

path = r'\\aafssrv\SHARED-BI\files_aa_dwh_x\dataBase\qldData\costs'

files = glob.glob(path + "/*.csv")
li = []
colNames =  ["product", "unit", "costNorm", "region", "group"]
for filename in files:
    temp_df = pd.read_csv(filename, sep="|", encoding="utf-8")#, index_col=None)#, header=0)
    temp_df.drop(["Unnamed: 0", "№"], axis=1, inplace=True)
    temp_df.columns = colNames
    li.append(temp_df)
costNorms = pd.concat(li, axis=0, ignore_index=True)
costNorms['region'] = costNorms['region'].replace("Шымкент г.а.",  "г.Шымкент", regex=True)
costNorms['region'] = costNorms['region'].replace("Алматы г.а.",  "г.Алматы", regex=True)
costNorms['region'] = costNorms['region'].replace("Нур-Султан г.а.",  "г.Нур-Султан", regex=True)
costNormsCodes_reg = pd.merge(costNorms, cato[["name", "cato_Id"]], left_on = "region", right_on = "name", how='left').drop(columns = ["name", "region"])
costNormsCodes_prod = pd.merge(costNormsCodes_reg, qName[["name", "codERP"]], left_on = "product", right_on = "name", how='left').drop(columns = ["name"])
costNormsCodes_prod.to_csv(r'\\aafssrv\SHARED-BI\files_aa_dwh_x\dataBase\qldData\costs\costNorms.csv', encoding='utf-8', sep='|', quoting = csv.QUOTE_NONE, escapechar = '')
TO_CHECK = costNormsCodes_prod[costNormsCodes_prod["codERP"].isna()]
TO_CHECK.to_excel(r"\\aafssrv\SHARED-BI\files_aa_dwh_x\dataBase\qldData\CHECK\TO_CHECK_COSTS.xlsx")

In [ ]:
print("YEAH YOU DID IT!")

In [ ]:
d.close()